In [1]:
!pip install pyiceberg pandas pyarrow sqlalchemy --quiet


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [2]:
warehouse_path = "iceberg"

In [3]:
!rm -r iceberg/*

In [4]:
import pandas as pd

In [5]:
pd.read_parquet('hacker_news.parquet').head()

,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,None,None,Could this be a replacement for git-annex?\nI&...,None,apex_sloth,NaN,1668589952,2022-11-16 09:12:32,comment,33620611,33612410.0,NaN,NaN,None
1,None,None,Wikipedia is definitely a good example of inte...,None,divan,NaN,1668589950,2022-11-16 09:12:30,comment,33620610,33613238.0,NaN,NaN,None
2,None,None,"&gt; HTML, XML, and ZIP files aren&#x27;t goin...",None,lqet,NaN,1668589936,2022-11-16 09:12:16,comment,33620609,33617161.0,NaN,NaN,None
3,None,None,It’s incredible and the more organisations doi...,None,randomsearch,NaN,1668589918,2022-11-16 09:11:58,comment,33620608,33620090.0,NaN,NaN,None
4,None,None,Hydrogen might be a solution to powering marin...,None,preisschild,NaN,1668589914,2022-11-16 09:11:54,comment,33620607,33618793.0,NaN,NaN,None


In [6]:
from pyiceberg.catalog.sql import SqlCatalog

catalog = SqlCatalog(
    "default",
    **{
        "uri": "sqlite:///iceberg/pyiceberg_catalog.db",
        "warehouse": f"file://{warehouse_path}",
    },
)

In [7]:
import pyarrow.parquet as pq

df = pq.read_table("hacker_news.parquet")

In [8]:
catalog.create_namespace("default")

In [9]:
table = catalog.create_table(
    "default.hacker_news",
    schema=df.schema,
)

In [10]:
table.append(df)

In [11]:
len(table.scan().to_arrow())

250000